In [1]:
import matplotlib.pyplot as plt
import logging
import pandas as pd
import numpy as np
import os

In [2]:
logging.basicConfig(
    filename= 'historico_de_acoes.log',           #  f"{os.path.basename(__file__).split('.')[0]}.log",
    filemode= 'a',
    format= '%(asctime)s - %(levelname)s -  %(funcName)s - %(message)s',
    level= logging.INFO,
    datefmt= '%d-%b-%y %H:%M:%S',
    force= True)

## Carregando dados

In [3]:
all_clients_original = pd.read_csv("/content/contatos.csv", sep=";", dtype=str)

In [4]:
all_clients_original.shape

(1085, 41)

In [5]:
all_clients_original.head()

,ID,Código,Nome,Fantasia,Endereço,Número,Complemento,Bairro,CEP,Cidade,...,CPF mãe,Segmento,Vendedor,Tipo contato,E-mail para envio NFe,Limite de crédito,Cliente desde,Próxima visita,Condição de pagamento,Regime tributário
0,15924469412,NaN,Lorena Cristina Araújo,NaN,Rua C117,NaN,NaN,Jardim América,74255380,Goiânia,...,NaN,NaN,Renata Wist,Cliente,NaN,0.00,04/11/2022,NaN,NaN,NaN
1,15924519125,NaN,Thais Barbosa,NaN,Alameda das Rosas,1429,esq R 17,Setor Oeste,74125010,Goiânia,...,NaN,NaN,NaN,Cliente,NaN,0.00,04/11/2022,NaN,NaN,NaN
2,15928798594,NaN,Cristiana Faccin,NaN,Rua 56,NaN,NaN,Jardim Goiás,74810240,Goiânia,...,NaN,NaN,NaN,Cliente,NaN,0.00,08/11/2022,NaN,NaN,NaN
3,15928820386,NaN,Zany Ferraz Atelier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Cliente,NaN,0.00,08/11/2022,NaN,NaN,NaN
4,15928822825,NaN,Glaubinha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Cliente,NaN,0.00,NaN,NaN,NaN,NaN


In [6]:
total_columns = len(all_clients_original.columns)
print(total_columns)

41


In [7]:
for column in all_clients_original.columns:
    print('\n', column)
    print(all_clients_original[column].value_counts(dropna=False))


 ID
15924469412    1
15931316194    1
15928728139    1
15928751591    1
15928782743    1
              ..
15922042993    1
15922042994    1
15922042995    1
15922042996    1
16031183985    1
Name: ID, Length: 1085, dtype: int64

 Código
NaN    1085
Name: Código, dtype: int64

 Nome
Layanne Braga                    3
Frete                            2
Patrícia Martins Botosso         2
IANA PUCCA                       2
Compra de acessórios             2
                                ..
Janene De Morais Bulman          1
Janicleia Sampaio                1
Jaqueline Da Silva Pinto         1
Jaqueline Setembrino Da Silva    1
Tatiana de Jesus Amaral          1
Name: Nome, Length: 1065, dtype: int64

 Fantasia
NaN                       956
@soniamariaalvesteles       2
@marryflor                  1
jak.sousa                   1
day_linda_de_bonita         1
                         ... 
marypinkmenegatti           1
@rita_gnoronha              1
suelybarcelosmeireles       1
@vanfsdemat

## Coisas pra fazer

- [ ]  Pegar os nomes que se repetem e analisar
- [ ]  Formatar os instagram
- [ ]  Observar os moradoras da Rua Dona Stela
- [ ]  Padronizar os "s/n"
- [ ]  Gerar um mapa por CEP
- [ ]  Mapa de calor por estado
- [ ]  Observar cadastros sem o estado, cidade, etc
- [ ]  Observar fone repetidos
- [ ]  Observar celular repetidos
- [ ]  Observar email repetidos
- [ ]  Observar pessoas jurídicas
- [ ]  Ativar situações
- [ ]  Observar sexo
- [ ]  Observar data nascimento e idade
- [ ]  Observar vendedor
- [ ]  Observar Tipo de cliente


### Pegar nomes que se repetem e analisar

- Vai passar por todos os nomes e pegar a quantidade de repetição de cada um
- Verificar qual cadastro está mais completo e excluir os outros

In [8]:
repetition_by_name = all_clients_original['Nome'].value_counts(dropna = False)

In [9]:
index_first_name_no_repeat = 0
while (repetition_by_name[index_first_name_no_repeat] > 1):
  index_first_name_no_repeat += 1

array_repeats_names = repetition_by_name.index[:index_first_name_no_repeat]

In [10]:
# Retorna o índice da Series com menos informações NaN e todos os indices dos outros
def registration_with_less_nan(df):
  less_quantity = total_columns
  idx_less_quantity = 0

  for idx, series in df.iterrows():
    counts_nan = series.isna().sum()

    if (counts_nan < less_quantity):
      less_quantity = counts_nan
      idx_less_quantity = idx

  return idx_less_quantity

In [11]:
def scraping_dates(idx_more_complete, df_registration_same_names):
  series_registration_more_complete = df_registration_same_names.loc[idx_more_complete]

  df_others_registration = df_registration_same_names.drop(idx_more_complete, axis=0)

  for data_type, value in series_registration_more_complete.iteritems():
    if (pd.isnull(value)):
      for idx_less_complete in df_others_registration.index:
        new_value = df_others_registration.loc[idx_less_complete][data_type]
        if (not(pd.isnull(new_value))):
          series_registration_more_complete[data_type] = new_value

          logging.info(f'{idx_less_complete} para {idx_more_complete} troca na categoria "{data_type}": de {value} para {new_value}')

In [12]:
for name in array_repeats_names:
  df_registration_same_names = all_clients_original.loc[all_clients_original["Nome"]==name].copy()

  idx_registration_more_complete = registration_with_less_nan(df_registration_same_names)
  
  scraping_dates(idx_registration_more_complete, df_registration_same_names)

  break